A small side project (from this already-side-project!) to look at the lowest recorded temperatures in the area. 
I am looking at installing a high-efficiency heat pump that only works when temperatures are above -13F (-25C). 
When temperatures are lower than that, the heat pump stops working and the house would have no heat at the worst possible time. 
So the obvious question is: How often does it get below (or near) that temperature?

I suspect this information is available online somewhere, but I've got all this tooling right here, so what the heck.

In [ ]:
import download_historical_data as dl
import os 
import matplotlib.pyplot as plt
import pandas as pd

plt.style.use("default")  # alternative "ggplot"

HISTORICAL_DATA_DIR = os.path.abspath("./historical_data")
WEATHER_DATA_DIR = os.path.join(HISTORICAL_DATA_DIR, "weather_station_data")
ANALYSIS_DATA_DIR = os.path.abspath("./analysis_data/")
LOW_TEMP_DATA_DIR = os.path.join(ANALYSIS_DATA_DIR, "low_temps")

for dir in [HISTORICAL_DATA_DIR, WEATHER_DATA_DIR, LOW_TEMP_DATA_DIR, ANALYSIS_DATA_DIR]:
    if not os.path.exists(dir):
        os.makedirs(dir)


WEATHER_STATION_IDS = [
    "USW00023066",  # Grand Junction Walker Field
    "USC00053553",  # Greeley UNC
    "USC00053005",  # Ft Collins
    "USC00050848",  # Boulder
    "USC00055984",  # Northglenn
    "USC00058995",  # Wheat Ridge
    "USW00023061"  # Alamosa
]

In [ ]:
# Uncomment to force re-download of source data
# Otherwise can also run the download script manually via: python download_historical_data.py
# Data files are saved locally so you only need to re-download to get new/different data

#dl.download_eia_historical_data(ELECTRIC_DATA_DIR, eia_respondent="PSCO")
#dl.download_ghcnd_historical_data(WEATHER_DATA_DIR, WEATHER_STATION_IDS)

In [ ]:
## Read in the weather DataFrame, but only for Denver-area weather stations
files = [
    os.path.join(WEATHER_DATA_DIR, "USC00053005-dataframe.json"),
    os.path.join(WEATHER_DATA_DIR, "USC00050848-dataframe.json"),
    os.path.join(WEATHER_DATA_DIR, "USC00055984-dataframe.json"),
    os.path.join(WEATHER_DATA_DIR, "USC00058995-dataframe.json"),
]
temp_df = dl.read_weather_data(files, earliest_date=None)
temp_df.head()

In [ ]:
for c in temp_df.columns:
    if c.endswith("_tmax"):
        temp_df.drop(c, axis="columns", inplace=True)

renames = {
    "USC00053005_tmin": "FtCollins",
    "USC00050848_tmin": "Boulder",
    "USC00055984_tmin": "Northglenn",
    "USC00058995_tmin": "WheatRidge"
}
temp_df.rename(columns=renames, inplace=True)
temp_df.head()

In [ ]:
stacked = temp_df.stack()
# stacked.columns.set_names(names="min_temp", inplace=True)
stacked.index.set_names(names="station", level=1, inplace=True)
stacked = pd.DataFrame(stacked)
stacked.rename(columns={0:"celsius"}, inplace=True)
stacked["fahrenheit"] = (stacked["celsius"] * 1.8) + 32
stacked.head()

In [ ]:
# Sort by date, latest->earliest
stacked.sort_index(axis=0, level=0, ascending=False, inplace=True)
stacked.head(10)

In [ ]:
sorted = stacked.sort_values(by="celsius", ascending=False, kind="stable")
sorted.head()

How many days get down to our cutoff temp of -25C?

In [ ]:
very_low_days = sorted[sorted["celsius"] <= -25]
len(very_low_days)

In [ ]:
very_low_days.head(25)


In [ ]:
## How many days get _close_ to our cutoff?
low_days = sorted[sorted["celsius"] < -20]
len(low_days)

In [ ]:
low_days.head(20)

In [ ]:
low_days.hist()

In [ ]:
## Record-breaking coldest days
rev_sorted = stacked.sort_values(by="celsius", ascending=True, kind="stable")
rev_sorted.head(25)